In [30]:
# pip install numpy==1.24

In [31]:
import numpy as np
import pandas as pd
import spacy
import nltk
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from spacy import displacy
from spacy.matcher import PhraseMatcher
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from spacy.lang.en import English

In [32]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kbeni\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\kbeni\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kbeni\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\kbeni\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\kbeni\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [33]:
data = pd.read_pickle('Data/mergerdata.pkl')
# data = pd.read_json('merger_data.json')

In [34]:
data

,date_completion,bidders_name,date_completion_assumed,date_last_status_update,status,bidders_industry,id,comments,targets_industry,targets_isin,date_rumor,date_postponed,date_announcement,bidders_isin,date_completion_expected,date_withdrawn,targets_country,type,targets_name,bidders_country
0,None,[],None,None,Rumour - Expired,[],1601040256,2008-11-26: Federal Property Management Agency...,[0811],[None],2008-11-26,None,None,[],None,None,[RU],Acquisition 100%,[ivanovskoe karyeroupravlenie oao],[]
1,None,[cmc centrozlom sp zoo],None,None,Rumour - Expired,[4672],1601232181,2010-11-10: Polish Treasury Ministry stated th...,[4672],[None],2010-11-10,None,None,[None],None,None,[PL],Acquisition increased from 51% to 92.72%,[scrapena sa],[PL]
2,None,[morningstar japan kk],2010-02-26,None,Completed Assumed,[6312],620622,2008-02-25: it was announced that Morningstar ...,[],[],2008-02-25,None,2008-02-25,[JP3922100007],None,None,[],Acquisition increased from 52.3% to 100%,[],[JP]
3,2023-04-03,[fabryka obrabiarek rafamet sa],None,2023-04-06,Completed,[2849],2007102004,2023-02-15: it was stated that Fabryka Obrabia...,[2451],[None],2023-02-15,None,2023-03-09,[PLRFMET00016],None,None,[PL],Acquisition increased from 100% to 100%,[odlewnia rafamet sp zoo],[PL]
4,2014-04-22,[fabrika kartona umka doo],None,None,Completed,[1712],1909107449,"2014-04-28: it was announced that on 22/04/14,...",[1712],[RSFHBGE53066],2014-04-22,None,2014-04-22,[RSUMKAE56492],None,None,[RS],Acquisition 64.131%,[fabrika hartije doo],[RS]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825768,2004-07-14,[taj gvk hotels and resorts ltd],None,2004-07-14,Completed,[5510],151050,2002-04-26: India Tourism Development Corp (IT...,[],[],2002-04-26,None,2002-05-03,[INE586B01026],None,None,[],Acquisition 100%,[],[IN]
825769,None,[],2021-06-13,None,Completed Assumed,[],1939068111,2019-06-13: it was announced that Biocannabix ...,[2120],[None],2019-06-13,None,2019-06-13,[],None,None,[CO],Acquisition 100%,[lifeline pharma sas],[]
825770,None,[mitutoyo corporation],2013-01-21,None,Completed Assumed,[2651],1633020959,2011-01-21: it was announced that Leeport Mach...,[4669],[None],2011-01-21,None,2011-01-21,[None],None,None,[VG],Acquisition increased from 10% to 51%,[leeport metrology corporation],[JP]
825771,None,[woori technology inc.],2010-04-22,None,Completed Assumed,[2712],1600005445,2008-04-22: it was announced that Woori Techno...,[6201],[None],2008-04-22,None,2008-04-22,[KR7032820003],None,None,[GB],Acquisition increased from 100% to 100%,"[mayhill co., ltd]",[KR]


In [35]:
companies = pd.read_csv("Data/DSB2.csv", sep=";")
companies

,Unnamed: 0,Company Common Name,TRBC Industry Name,TRBC Industry Name.1,ISIN,RIC,30.06.2022,30.09.2022,31.12.2022,31.03.2023,30.06.2023,30.09.2023,31.12.2023,31.03.2024,30.06.2024,30.09.2024
0,CIRI.MI,CIR SpA - Compagnie Industriali Riunite,"Auto, Truck & Motorcycle Parts","Auto, Truck & Motorcycle Parts",IT0000070786,CIRI.MI,"0,42242301","0,38657142","0,465004281","0,417845413","0,412943628","0,427029712","0,475097392","0,578184329","0,637420322","0,637978066"
1,STLAM.MI,Stellantis NV,Auto & Truck Manufacturers,Auto & Truck Manufacturers,NL00150001Q9,STLAM.MI,"12,36032788","12,02535987","14,19520548","18,14023564","17,55419544","19,24804718","23,34102171","28,41840697","19,78681236","13,8150643"
2,PNNI.MI,Pininfarina SpA,Auto & Truck Manufacturers,Auto & Truck Manufacturers,IT0003056386,PNNI.MI,"0,84484602","0,83095707","0,896832192","0,890969987","0,920804285","0,866743476","0,871839582","0,837072834","0,812041352","0,837276624"
3,XTC.TO,Exco Technologies Ltd,"Auto, Truck & Motorcycle Parts","Auto, Truck & Motorcycle Parts",CA30150P1099,XTC.TO,"6,207271597","5,569217416","5,683076242","5,660377358","6,029011786","5,472490241","5,949414874","5,480868666","5,812678219","5,90801538"
4,CONG.DE,Continental AG,"Auto, Truck & Motorcycle Parts","Auto, Truck & Motorcycle Parts",DE0005439004,CONG.DE,"69,70503763","45,02650635","59,91010274","74,70273903","75,38812337","70,56560297","84,88848179","72,16517087","56,67148749","64,73306241"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320,SHA0.DE,Schaeffler AG,"Auto, Truck & Motorcycle Parts","Auto, Truck & Motorcycle Parts",DE000SHA0019,SHA0.DE,"5,660258695","4,534987408","6,811857877","7,603594229","6,158696909","5,765958121","6,174610707","6,74188816","5,758208795","5,159494517"
1321,SHA0.DE,NaN,NaN,NaN,DE000SHA0019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1322,6988.TW,Wellysun Inc,"Auto, Truck & Motorcycle Parts","Auto, Truck & Motorcycle Parts",TW0006988009,6988.TW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1323,SAG.OQ,SAG Holdings Ltd,"Auto, Truck & Motorcycle Parts","Auto, Truck & Motorcycle Parts",KYG7776G1047,SAG.OQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# filter data from before 2010-01-01
data = data[data['date_rumor'] >= '2010-01-01']

In [37]:
ISIN = companies["ISIN"].unique()
# check now for entries in data[targets_isin] and data[bidders_isin] that have an element from ISIN
# Check for entries in data['targets_isin'] that have an element from ISIN
targets_isin_matches = data[data['targets_isin'].isin(ISIN)]

# Check for entries in data['bidders_isin'] that have an element from ISIN
bidders_isin_matches = data[data['bidders_isin'].isin(ISIN)]

# Display the results
print("Targets ISIN Matches:")
print(targets_isin_matches)

print("\nBidders ISIN Matches:")
print(bidders_isin_matches)

Targets ISIN Matches:
Empty DataFrame
Columns: [date_completion, bidders_name, date_completion_assumed, date_last_status_update, status, bidders_industry, id, comments, targets_industry, targets_isin, date_rumor, date_postponed, date_announcement, bidders_isin, date_completion_expected, date_withdrawn, targets_country, type, targets_name, bidders_country]
Index: []

Bidders ISIN Matches:
Empty DataFrame
Columns: [date_completion, bidders_name, date_completion_assumed, date_last_status_update, status, bidders_industry, id, comments, targets_industry, targets_isin, date_rumor, date_postponed, date_announcement, bidders_isin, date_completion_expected, date_withdrawn, targets_country, type, targets_name, bidders_country]
Index: []


In [38]:
import ast

# Function to check if any relevant ISIN is included
def compare_if_included(isin_list):
    if not isin_list:  # Handle empty or None values gracefully
        return False
    try:
        # Parse the entry if it's a string representation of a list
        if isinstance(isin_list, str):
            isin_list = ast.literal_eval(isin_list)
        # Check if any ISIN matches the relevant ISINs
        return any(str(isin) in ISIN for isin in isin_list)
    except (ValueError, SyntaxError, TypeError):
        # Return False if parsing fails or isin_list is malformed
        return False

# Apply the function to the 'targets_isin' and 'bidders_isin' columns
data['relevant_target'] = data['targets_isin'].apply(compare_if_included)
data['relevant_bidder'] = data['bidders_isin'].apply(compare_if_included)

# Display the results
print("Targets ISIN Matches:")
print(data[data['relevant_target']])

print("\nBidders ISIN Matches:")
print(data[data['relevant_bidder']])

C:\Users\kbeni\AppData\Local\Temp\ipykernel_14316\4158054007.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['relevant_target'] = data['targets_isin'].apply(compare_if_included)


Targets ISIN Matches:
       date_completion                                       bidders_name  \
17          2014-07-31                    [jaya hind investments pvt ltd]   
300               None                    [china automotive holdings ltd]   
321               None                          [billion legend co., ltd]   
3368        2013-09-12  [him teknoforge ltd, globe precision industrie...   
3945        2012-04-14                [oriental carbon and chemicals ltd]   
...                ...                                                ...   
818453      2015-07-13      [shanghai lianming investment group co., ltd]   
820672      2018-12-26                          [alliance rostec auto bv]   
821227            None                                                 []   
824212            None                                     [tatshina ooo]   
825541            None                            [b4s solutions pvt ltd]   

       date_completion_assumed date_last_status_updat

C:\Users\kbeni\AppData\Local\Temp\ipykernel_14316\4158054007.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['relevant_bidder'] = data['bidders_isin'].apply(compare_if_included)


In [39]:
# Filter data to only include rows with relevant codes
data = data[(data['relevant_target']) | (data['relevant_bidder'])]

In [40]:
data.drop(columns=['relevant_bidder', 'relevant_target'], inplace=True)
data

,date_completion,bidders_name,date_completion_assumed,date_last_status_update,status,bidders_industry,id,comments,targets_industry,targets_isin,date_rumor,date_postponed,date_announcement,bidders_isin,date_completion_expected,date_withdrawn,targets_country,type,targets_name,bidders_country
17,2014-07-31,[jaya hind investments pvt ltd],None,None,Completed,[6499],1907056319,2014-08-01: it was announced that on 31/07/14 ...,[2910],[INE451A01017],2014-07-31,None,2014-07-31,[None],None,None,[IN],Acquisition 3.188%,[force motors ltd],[IN]
162,2022-07-22,"[cwb automotive electronics co., ltd]",None,None,Completed,[2931],1943340721,2022-04-27: it was announced that CWB Automoti...,[2561],[None],2022-04-27,None,2022-04-27,[CNE100004C11],None,None,[CN],Acquisition increased from 100% to 100%,"[yueqing guanghe surface treatment co., ltd]",[CN]
238,None,"[beijing bdstar navigation co., ltd]",2021-09-16,None,Completed Assumed,[2651],1943132402,2019-09-16: it was announced that Beijing BDSt...,[4669],[None],2019-09-16,None,2019-09-16,[CNE100000643],None,None,[CN],Acquisition increased from 80% to 90%,[beijing beidou xingtong positioning technolog...,[CN]
259,2015-06-02,"[patec precision industry co., ltd]",None,None,Completed,[2932],1907102483,2015-06-02: it was announced that Patec Precis...,[2932],[None],2015-06-02,None,2015-06-02,[KYG693691092],None,None,[SG],Acquisition increased from 100% to 100%,[patec pte ltd],[KY]
300,None,[china automotive holdings ltd],None,2021-01-27,Withdrawn,[4519],1941252521,2019-04-23: it was announced that Sprintex Ltd...,[2931],[AU000000SIX0],2019-04-23,None,2019-04-23,[None],None,2020-08-28,[AU],Acquisition unknown stake %,[sprintex ltd],[CN]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824785,None,"[changzhou tenglong auto parts co., ltd]",2018-03-23,None,Completed Assumed,[2930],1907148524,2016-03-23: it was announced that Changzhou Te...,[2932],[None],2016-03-23,None,2016-03-23,[CNE1000022D8],None,None,[CN],Acquisition increased from 100% to 100%,"[xiamen dajun accurate industrial co., ltd]",[CN]
824819,2017-12-06,"[zhejiang tiantie industry co., ltd]",None,2017-12-08,Completed,[2219],1943003558,2017-11-20: it was announced that Zhejiang Tia...,[2899],[None],2017-11-20,None,2017-11-20,[CNE100002F77],None,None,[CN],Acquisition 51%,"[hebei lutong railway equipment co., ltd]",[CN]
825015,2011-09-30,[autech corporation],None,None,Completed,[2910],1633048164,2011-09-30: it was announced that Autech Corpo...,[2521],[None],2011-09-30,None,2011-09-30,[KR7067170001],None,None,[KR],Acquisition 50.1%,[clk corporation],[KR]
825067,None,"[dongfeng automobile co., ltd]",2013-10-18,None,Completed Assumed,[2910],1633049890,2011-10-18: it was announced that Dongfeng Aut...,[],[],2011-10-18,None,2011-10-18,[CNE000000ZT3],None,None,[],Acquisition 100%,[],[CN]


In [41]:
# # filter industries by keywords
# industries_data = pd.read_csv('Data/industries.csv')
#
# keywords_auto_oil = ["automobile", "vehicle", "car", "motor", "engine", "oil", "gas", "fuel", "petroleum"]
# relevant_rows = industries_data[
#     industries_data['description'].str.contains('|'.join(keywords_auto_oil), case=False, na=False)
# ]
#
# # Extract the unique codes for these descriptions
# relevant_codes = relevant_rows['code'].unique()
#
# # change relevant_codes to strings
# relevant_codes = [str(code) for code in relevant_codes]
#
# relevant_codes = set(relevant_codes)

In [42]:
# import ast
#
#
# # Function to check if any relevant code is included
# def compare_if_included(industries):
#     if not industries:  # Handle empty or None values gracefully
#         return False
#     try:
#         # Parse the entry if it's a string representation of a list
#         if isinstance(industries, str):
#             industries = ast.literal_eval(industries)
#         # Check if any industry code matches the relevant codes
#         return any(str(industry) in relevant_codes for industry in industries)
#     except (ValueError, SyntaxError, TypeError):
#         # Return False if parsing fails or industries is malformed
#         return False
#
# # Apply the function to the 'targets_industry' column
# data['has_relevant_code'] = data['targets_industry'].apply(compare_if_included)

In [43]:
# # Filter data to only include rows with relevant codes
# data = data[data['has_relevant_code']]

In [44]:
# data.drop(columns=['has_relevant_code'], inplace=True)

In [45]:
# Step 3: Sentence Tokenization (using NLTK)
data['sentences'] = data['comments'].apply(lambda x: sent_tokenize(x))

In [46]:
# Step 4: Lowercasing and Removing Punctuation
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation (SpaCy already handles punctuation during processing)
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])
    
    return text

In [47]:
# Apply cleaning function to each sentence
data['cleaned_sentences'] = data['sentences'].apply(lambda sentences: [clean_text(sentence) for sentence in sentences])

In [48]:
# Step 5: Stemming (optional, more aggressive than Lemmatization)
def apply_stemming(sentence):
    ps = PorterStemmer()
    words = word_tokenize(sentence)
    return ' '.join([ps.stem(word) for word in words])

In [49]:
# Apply stemming to each sentence
data['stemmed_sentences'] = data['cleaned_sentences'].apply(lambda sentences: [apply_stemming(sentence) for sentence in sentences])

In [50]:
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [51]:
nlp = spacy.load('en_core_web_sm')

In [52]:
# Step 6: Lemmatization using SpaCy
def lemmatize_sentence(sentence):
    doc = nlp(sentence)
    return ' '.join([token.lemma_ for token in doc])

In [53]:
# Apply Lemmatization
data['lemmatized_sentences'] = data['cleaned_sentences'].apply(lambda sentences: [lemmatize_sentence(sentence) for sentence in sentences])

In [54]:
data.head()

,date_completion,bidders_name,date_completion_assumed,date_last_status_update,status,bidders_industry,id,comments,targets_industry,targets_isin,...,date_completion_expected,date_withdrawn,targets_country,type,targets_name,bidders_country,sentences,cleaned_sentences,stemmed_sentences,lemmatized_sentences
17,2014-07-31,[jaya hind investments pvt ltd],None,None,Completed,[6499],1907056319,2014-08-01: it was announced that on 31/07/14 ...,[2910],[INE451A01017],...,None,None,[IN],Acquisition 3.188%,[force motors ltd],[IN],[2014-08-01: it was announced that on 31/07/14...,[20140801 it was announced that on 310714 jaya...,[20140801 it wa announc that on 310714 jaya hi...,[20140801 it be announce that on 310714 jaya h...
162,2022-07-22,"[cwb automotive electronics co., ltd]",None,None,Completed,[2931],1943340721,2022-04-27: it was announced that CWB Automoti...,[2561],[None],...,None,None,[CN],Acquisition increased from 100% to 100%,"[yueqing guanghe surface treatment co., ltd]",[CN],[2022-04-27: it was announced that CWB Automot...,[20220427 it was announced that cwb automotive...,[20220427 it wa announc that cwb automot elect...,[20220427 it be announce that cwb automotive e...
238,None,"[beijing bdstar navigation co., ltd]",2021-09-16,None,Completed Assumed,[2651],1943132402,2019-09-16: it was announced that Beijing BDSt...,[4669],[None],...,None,None,[CN],Acquisition increased from 80% to 90%,[beijing beidou xingtong positioning technolog...,[CN],[2019-09-16: it was announced that Beijing BDS...,[20190916 it was announced that beijing bdstar...,[20190916 it wa announc that beij bdstar navig...,[20190916 it be announce that beijing bdstar n...
259,2015-06-02,"[patec precision industry co., ltd]",None,None,Completed,[2932],1907102483,2015-06-02: it was announced that Patec Precis...,[2932],[None],...,None,None,[SG],Acquisition increased from 100% to 100%,[patec pte ltd],[KY],[2015-06-02: it was announced that Patec Preci...,[20150602 it was announced that patec precisio...,[20150602 it wa announc that patec precis indu...,[20150602 it be announce that patec precision ...
300,None,[china automotive holdings ltd],None,2021-01-27,Withdrawn,[4519],1941252521,2019-04-23: it was announced that Sprintex Ltd...,[2931],[AU000000SIX0],...,None,2020-08-28,[AU],Acquisition unknown stake %,[sprintex ltd],[CN],[2019-04-23: it was announced that Sprintex Lt...,[20190423 it was announced that sprintex ltd a...,[20190423 it wa announc that sprintex ltd a ma...,[20190423 it be announce that sprintex ltd a m...


In [55]:
for text in data['lemmatized_sentences']:
    print(text)

['20140801 it be announce that on 310714 jaya hind investments pvt ltd and other promoter have acquire a total of 420000 share in force motors ltd a pune indiabase automobile manufacturer', 'thus the total consideration could be value at inr 222936 million base on the closing share price of inr 5308 per share on 300714 the last trading date prior to the transaction', 'base on force motors share capital of 13176262 share the acquire share represent a 3188 per cent stake in the company', 'upon completion jaya hind investment be hold 7869411 share or 59724 per cent of force motor', 'the transaction also include the sale of force motor patent', 'no further detail be disclose']
['20220427 it be announce that cwb automotive electronics co ltd have agree to inject capital in its wholly own subsidiary yueqe guanghe surface treatment co ltd a yueqing chinabase metal surface and heat treatment service company', 'the consideration be cny 25 million', 'on completion the ownership will remain uncha

In [56]:
# Step 7: Part-of-Speech (POS) Tagging using NLTK (optional, more for understanding structure)
def pos_tagging(sentence):
    return pos_tag(word_tokenize(sentence))

In [57]:
# Apply POS tagging
data['pos_tagged_sentences'] = data['cleaned_sentences'].apply(lambda sentences: [pos_tagging(sentence) for sentence in sentences])

In [58]:
# Step 8: Dependency Parsing using SpaCy
def dependency_parsing(sentence):
    doc = nlp(sentence)
    return [(token.text, token.dep_, token.head.text) for token in doc]

In [59]:
# Apply Dependency Parsing
data['dependency_parsed_sentences'] = data['cleaned_sentences'].apply(lambda sentences: [dependency_parsing(sentence) for sentence in sentences])

In [60]:
data.head()

,date_completion,bidders_name,date_completion_assumed,date_last_status_update,status,bidders_industry,id,comments,targets_industry,targets_isin,...,targets_country,type,targets_name,bidders_country,sentences,cleaned_sentences,stemmed_sentences,lemmatized_sentences,pos_tagged_sentences,dependency_parsed_sentences
17,2014-07-31,[jaya hind investments pvt ltd],None,None,Completed,[6499],1907056319,2014-08-01: it was announced that on 31/07/14 ...,[2910],[INE451A01017],...,[IN],Acquisition 3.188%,[force motors ltd],[IN],[2014-08-01: it was announced that on 31/07/14...,[20140801 it was announced that on 310714 jaya...,[20140801 it wa announc that on 310714 jaya hi...,[20140801 it be announce that on 310714 jaya h...,"[[(20140801, CD), (it, PRP), (was, VBD), (anno...","[[(20140801, npadvmod, announced), (it, nsubjp..."
162,2022-07-22,"[cwb automotive electronics co., ltd]",None,None,Completed,[2931],1943340721,2022-04-27: it was announced that CWB Automoti...,[2561],[None],...,[CN],Acquisition increased from 100% to 100%,"[yueqing guanghe surface treatment co., ltd]",[CN],[2022-04-27: it was announced that CWB Automot...,[20220427 it was announced that cwb automotive...,[20220427 it wa announc that cwb automot elect...,[20220427 it be announce that cwb automotive e...,"[[(20220427, CD), (it, PRP), (was, VBD), (anno...","[[(20220427, npadvmod, announced), (it, nsubjp..."
238,None,"[beijing bdstar navigation co., ltd]",2021-09-16,None,Completed Assumed,[2651],1943132402,2019-09-16: it was announced that Beijing BDSt...,[4669],[None],...,[CN],Acquisition increased from 80% to 90%,[beijing beidou xingtong positioning technolog...,[CN],[2019-09-16: it was announced that Beijing BDS...,[20190916 it was announced that beijing bdstar...,[20190916 it wa announc that beij bdstar navig...,[20190916 it be announce that beijing bdstar n...,"[[(20190916, CD), (it, PRP), (was, VBD), (anno...","[[(20190916, npadvmod, announced), (it, nsubjp..."
259,2015-06-02,"[patec precision industry co., ltd]",None,None,Completed,[2932],1907102483,2015-06-02: it was announced that Patec Precis...,[2932],[None],...,[SG],Acquisition increased from 100% to 100%,[patec pte ltd],[KY],[2015-06-02: it was announced that Patec Preci...,[20150602 it was announced that patec precisio...,[20150602 it wa announc that patec precis indu...,[20150602 it be announce that patec precision ...,"[[(20150602, CD), (it, PRP), (was, VBD), (anno...","[[(20150602, npadvmod, announced), (it, nsubjp..."
300,None,[china automotive holdings ltd],None,2021-01-27,Withdrawn,[4519],1941252521,2019-04-23: it was announced that Sprintex Ltd...,[2931],[AU000000SIX0],...,[AU],Acquisition unknown stake %,[sprintex ltd],[CN],[2019-04-23: it was announced that Sprintex Lt...,[20190423 it was announced that sprintex ltd a...,[20190423 it wa announc that sprintex ltd a ma...,[20190423 it be announce that sprintex ltd a m...,"[[(20190423, CD), (it, PRP), (was, VBD), (anno...","[[(20190423, npadvmod, announced), (it, nsubjp..."


In [65]:
data.to_json('Data/prepro__mergerdata.json')

In [66]:
data.to_csv('Data/prepro_mergerdata.csv')

In [63]:
for bidder in data['bidders_name']:
    print(bidder)

['jaya hind investments pvt ltd']
['cwb automotive electronics co., ltd']
['beijing bdstar navigation co., ltd']
['patec precision industry co., ltd']
['china automotive holdings ltd']
['billion legend co., ltd']
['yokohama rubber co., ltd, the']
['dometic group ab']
['sollers oao']
['zhejiang yinlun machinery co., ltd']
['beijing bdstar navigation co., ltd']
['exide industries ltd']
['cenntro electric group ltd']
['ubivelox inc.']
['iochpe-maxion sa']
['brisa bridgestone sabanci lastik sanayi ve ticaret as']
['indomobil sukses internasional tbk, pt', 'tritunggal intipermata, pt']
['dongyang piston co., ltd']
['ikka holdings (cayman) ltd']
['noblelift equipment joint stock co., ltd']
['thn corporation']
['tokyo radiator mfg co., ltd', 'selamat sempurna tbk, pt']
['him teknoforge ltd', 'globe precision industries pvt ltd']
['brembo spa']
['ford motor company']
['chengdu qianfeng electronics co., ltd']
['oriental carbon and chemicals ltd']
['paragon gmbh & co. kgaa']
['amerigon inc.']
['

In [64]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3395 entries, 17 to 825541
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   date_completion              1548 non-null   object
 1   bidders_name                 3395 non-null   object
 2   date_completion_assumed      1178 non-null   object
 3   date_last_status_update      973 non-null    object
 4   status                       3395 non-null   object
 5   bidders_industry             3395 non-null   object
 6   id                           3395 non-null   int64 
 7   comments                     3395 non-null   object
 8   targets_industry             3395 non-null   object
 9   targets_isin                 3395 non-null   object
 10  date_rumor                   3395 non-null   object
 11  date_postponed               2 non-null      object
 12  date_announcement            2964 non-null   object
 13  bidders_isin                 3395 n